In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import re
from collections import deque

In [3]:
player_decks = {}

with open('data/input_day22.txt', 'r') as f :
    for line in f.readlines():
        line = line.strip().replace(':','')
        if 'Player' in line:
            player=line
            player_decks[player] = deque([])
        elif len(line) < 1: 
            continue
        else:
            player_decks[player].append(int(line))
        
        

In [9]:
while (len(player_decks['Player 1'])>0) and (len(player_decks['Player 2'])>0):
    p1 = player_decks['Player 1'].popleft()
    p2 = player_decks['Player 2'].popleft()
    
    if p1 > p2:
        player_decks['Player 1'].extend([p1,p2])
    if p2 > p1:
        player_decks['Player 2'].extend([p2,p1])
        

In [22]:
def score_deck(deck):
    score_deck = deck.copy()
    multiplier=1
    score=0
    while len(score_deck) > 0:
        card = score_deck.pop()
        score += card*multiplier
        multiplier +=1
    return score

In [23]:
score_deck(player_decks['Player 1'])

6341

# Recursive War

In [24]:
from copy import deepcopy

In [29]:
def create_string(deck_dictionary):
    string = 'p1_'+'_'.join([str(i) for i in deck_dictionary['Player 1']])
    string +='_p2_'+'_'.join([str(i) for i in deck_dictionary['Player 2']])
    return string

def play_war_recurse(deck_dictionary ):
    round_strings=[]
    winner=None
    
    while (winner is None):
        deck_str = create_string(deck_dictionary)
        if  deck_str in round_strings:
            winner = 'Player 1'
            break
        round_strings.append(deck_str)
        
        p1 = deck_dictionary['Player 1'].popleft()
        p2 = deck_dictionary['Player 2'].popleft()
        
        if (   (p1 > len(deck_dictionary['Player 1']))
            or (p2 > len(deck_dictionary['Player 2'])) ):
            
            if p1 > p2:
                deck_dictionary['Player 1'].extend([p1,p2])
            if p2 > p1:
                deck_dictionary['Player 2'].extend([p2,p1])
            
            if len(deck_dictionary['Player 2'])==0:
                winner ='Player 1'
                break
            if len(deck_dictionary['Player 1'])==0:
                winner ='Player 2'
                break
        else:
            sub_game = {
                'Player 1' : deque(list(deck_dictionary['Player 1'])[:p1]),
                'Player 2' : deque(list(deck_dictionary['Player 2'])[:p2]),
            }
            round_winner, _ = play_war_recurse(sub_game)
            
            if round_winner == 'Player 1':
                deck_dictionary['Player 1'].extend([p1,p2])
            else:
                deck_dictionary['Player 2'].extend([p2,p1])
            
            if len(deck_dictionary['Player 2'])==0:
                winner ='Player 1'
                break
            if len(deck_dictionary['Player 1'])==0:
                winner ='Player 2'
                break
    return winner, deck_dictionary[winner]
    

In [30]:
test_deck = {
    'Player 1': deque([9,2,6,3,1]),
    'Player 2': deque([5,8,4,7,10]),
}


#test_deck = {
#    'Player 1': deque([43,19]),
#    'Player 2': deque([2,29,14]),
#}

In [31]:
winner, deck = play_war_recurse(deepcopy(test_deck))
score_deck(deck)

291

In [32]:
%%time
winner, deck = play_war_recurse(deepcopy(player_decks))
score_deck(deck)

CPU times: user 12 s, sys: 26.3 ms, total: 12 s
Wall time: 12 s


29177